# Generatiivsed võrgud

Korduvad närvivõrgud (RNN-id) ja nende väratiga rakutüübid, nagu Long Short Term Memory Cells (LSTM-id) ja Gated Recurrent Units (GRU-d), pakuvad mehhanismi keele modelleerimiseks, st nad suudavad õppida sõnade järjestust ja ennustada järgmise sõna järjestuses. See võimaldab meil kasutada RNN-e **generatiivseteks ülesanneteks**, nagu tavaline tekstigeneratsioon, masintõlge ja isegi pildiallkirjade loomine.

RNN arhitektuuris, mida käsitlesime eelmises osas, genereeris iga RNN üksus järgmise varjatud oleku väljundina. Kuid me saame lisada igale korduva üksusele veel ühe väljundi, mis võimaldab meil luua **järjestuse** (mis on sama pikk kui algne järjestus). Lisaks saame kasutada RNN üksusi, mis ei võta igal sammul sisendit, vaid kasutavad ainult algset olekuvektorit ja seejärel genereerivad väljundite järjestuse.

Selles märkmikus keskendume lihtsatele generatiivsetele mudelitele, mis aitavad meil teksti genereerida. Lihtsuse huvides loome **tähemärgi tasemel võrgu**, mis genereerib teksti täht-tähelt. Treeningu ajal peame võtma mõne tekstikorpuse ja jagama selle tähemärkide järjestusteks.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## Tegelaskujude sõnavara loomine

Tegelaskujude tasemel generatiivse võrgu loomiseks peame teksti jagama üksikuteks tähtedeks, mitte sõnadeks. Seda saab teha, määratledes teistsuguse tokeniseerija:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


Vaatame näidet, kuidas saame oma andmekogumist teksti kodeerida:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## Generatiivse RNN-i treenimine

RNN-i treenimiseks teksti genereerimiseks toimime järgmiselt. Igal sammul võtame `nchars` pikkuse tähemärkide jada ja palume võrgul genereerida iga sisendtähemärgi jaoks järgmine väljundtähemärk:

![Pilt, mis näitab näidet RNN-i genereerimisest sõnaga 'HELLO'.](../../../../../translated_images/et/rnn-generate.56c54afb52f9781d.webp)

Olenevalt konkreetsest stsenaariumist võime soovida lisada ka erimärke, näiteks *järjestuse lõpp* `<eos>`. Meie puhul tahame lihtsalt treenida võrku lõputu teksti genereerimiseks, seega määrame iga jada suuruseks `nchars` tokenit. Järelikult koosneb iga treeningnäide `nchars` sisendist ja `nchars` väljundist (mis on sisendjada, nihutatud ühe sümboli võrra vasakule). Minipartii koosneb mitmest sellisest jadast.

Minipartiide genereerimiseks võtame iga uudisteksti pikkusega `l` ja loome sellest kõik võimalikud sisend-väljund kombinatsioonid (neid kombinatsioone on `l-nchars`). Need moodustavad ühe minipartii ja minipartiide suurus on igal treenimissammul erinev.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

Nüüd määratleme generaatori võrgu. See võib põhineda mis tahes korduvrakul, mida arutasime eelmises osas (lihtne, LSTM või GRU). Meie näites kasutame LSTM-i.

Kuna võrk võtab sisendiks tähemärke ja sõnavara suurus on üsna väike, ei ole meil vaja sisendkihtide jaoks embedding-kihte; ühekuumkoodiga (one-hot-encoded) sisend võib otse minna LSTM-rakku. Kuid kuna me edastame tähemärkide numbreid sisendina, peame need enne LSTM-i edastamist ühekuumkoodiga kodeerima. Seda tehakse `forward` läbimise ajal, kutsudes `one_hot` funktsiooni. Väljundkooder oleks lineaarne kiht, mis teisendab peidetud oleku ühekuumkoodiga väljundiks.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

Treeningu ajal soovime olla võimelised genereeritud teksti näidiseid võtma. Selleks määratleme funktsiooni `generate`, mis loob väljundstringi pikkusega `size`, alustades algstringist `start`.

Selle tööpõhimõte on järgmine. Kõigepealt edastame kogu algstringi läbi võrgu ja võtame väljundoleku `s` ning järgmise ennustatud tähemärgi `out`. Kuna `out` on ühekujulise kodeeringuga, võtame `argmax`, et saada tähemärgi indeks `nc` sõnavaras, ning kasutame `itos`, et leida tegelik tähemärk ja lisada see tulemuseks olevate tähemärkide loendisse `chars`. Seda ühe tähemärgi genereerimise protsessi korratakse `size` korda, et luua vajalik arv tähemärke.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

Nüüd alustame treenimist! Treeningtsükkel on peaaegu sama nagu kõigis meie varasemates näidetes, kuid täpsuse asemel prindime iga 1000 epohhi järel genereeritud teksti näidise.

Erilist tähelepanu tuleb pöörata sellele, kuidas me arvutame kaotust. Kaotuse arvutamiseks vajame üheselt kodeeritud väljundit `out` ja oodatud teksti `text_out`, mis on tähemärkide indeksite loend. Õnneks ootab funktsioon `cross_entropy` esimeseks argumendiks normaliseerimata võrgu väljundit ja teiseks klassi numbrit, mis on täpselt see, mis meil olemas on. Lisaks teeb see automaatse keskmistamise minibatch'i suuruse järgi.

Samuti piirame treenimist `samples_to_train` näidiste arvuga, et mitte liiga kaua oodata. Soovitame teil katsetada ja proovida pikemat treenimist, võimalusel mitme epohhi jooksul (sellisel juhul peaksite selle koodi ümber looma uue tsükli).


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

See näide genereerib juba üsna head teksti, kuid seda saab mitmel viisil veelgi paremaks muuta:
* **Parem minibatchide genereerimine**. Viis, kuidas me treeningu jaoks andmeid ette valmistasime, oli ühe minibatchi genereerimine ühest proovist. See pole ideaalne, kuna minibatchid on kõik erineva suurusega ja mõnda neist ei saa isegi genereerida, kuna tekst on väiksem kui `nchars`. Lisaks ei koorma väikesed minibatchid GPU-d piisavalt. Targem oleks võtta üks suur tekstilõik kõigist proovidest, seejärel genereerida kõik sisend-väljund paarid, segada need ja luua võrdse suurusega minibatchid.
* **Mitmekihiline LSTM**. Tasub proovida 2 või 3 kihti LSTM-rakke. Nagu mainisime eelmises osas, eraldab iga LSTM kiht tekstist teatud mustreid, ja tähemärgi tasemel generaatori puhul võime eeldada, et madalam LSTM tase vastutab silpide eraldamise eest, kõrgemad tasemed aga sõnade ja sõnakombinatsioonide eest. Seda saab lihtsalt rakendada, andes LSTM konstruktorile kihtide arvu parameetri.
* Võid samuti katsetada **GRU üksustega** ja vaadata, millised annavad paremaid tulemusi, ning **erinevate varjatud kihtide suurustega**. Liiga suur varjatud kiht võib viia üleõppimisele (näiteks õpib võrk täpse teksti ära), ja väiksem suurus ei pruugi anda häid tulemusi.


## Pehme teksti genereerimine ja temperatuur

Eelmises `generate` definitsioonis valisime alati järgmise tähemärgina selle, millel oli suurim tõenäosus. See viis sageli selleni, et tekst "kordas" samu tähemärkide järjestusi uuesti ja uuesti, nagu selles näites:
```
today of the second the company and a second the company ...
```

Kui aga vaatame järgmise tähemärgi tõenäosusjaotust, võib juhtuda, et mõne kõrgeima tõenäosuse vahe ei ole suur, näiteks ühel tähemärgil võib olla tõenäosus 0.2 ja teisel 0.19 jne. Näiteks, kui otsime järgmist tähemärki järjestuses '*play*', võib järgmine tähemärk sama hästi olla kas tühik või **e** (nagu sõnas *player*).

See viib meid järelduseni, et alati ei ole "õiglane" valida tähemärki, millel on suurem tõenäosus, sest teise kõrgeima valimine võib siiski viia tähendusliku tekstini. Mõistlikum on **valida juhuslikult** tähemärke tõenäosusjaotusest, mille annab võrgu väljund.

Seda juhuslikku valimist saab teha `multinomial` funktsiooni abil, mis rakendab nn **multinomiaalset jaotust**. Funktsioon, mis rakendab seda **pehmet** teksti genereerimist, on defineeritud allpool:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

Oleme lisanud veel ühe parameetri, mida nimetatakse **temperatuuriks**, ja seda kasutatakse näitamaks, kui tugevalt peaksime järgima kõrgeimat tõenäosust. Kui temperatuur on 1.0, teeme õiglast multinomiaalset valimit, ja kui temperatuur läheneb lõpmatusele - muutuvad kõik tõenäosused võrdseks ning me valime järgmise tähemärgi juhuslikult. Allolevas näites näeme, et tekst muutub mõttetuks, kui temperatuuri liiga palju suurendame, ja meenutab "tsüklilist" rangelt genereeritud teksti, kui see läheneb 0-le.



---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
